In [1]:
from pddl import parse_domain, parse_problem

DOMAIN_PATH = 'pddl_files/domain.pddl'
PROBLEM_PATH = 'pddl_files/problem0.pddl'

domain = parse_domain(DOMAIN_PATH)
problem = parse_problem(PROBLEM_PATH)

In [46]:
constant_predicates, initial_predicates = [], []

for predicate in problem.init:
    if predicate.name in ["adjacent", "obstacle"]:
        constant_predicates.append(predicate)
    else:
        initial_predicates.append(predicate)

initial_predicates

[Predicate(munition, ), Predicate(at, pos62)]

In [32]:
all_actions = domain.actions

for action in domain.actions:
    print(action.precondition)

(and (adjacent ?pos1 ?pos2) (at ?pos1) (not (wall ?pos2)) (not (obstacle ?pos2)))
(and (adjacent ?pos1 ?pos2) (at ?pos1) (not (wall ?pos2)) (munition))


In [49]:
def get_neighbors(current_state, constant_predicates):
    states = []
    for action in get_actions(current_state, constant_predicates):
        pass
    return states

def get_actions(current_predicates, constant_predicates):
    actions = []
    for action in all_actions:
        for precond in action.precondition.operands:
            if precond not in current_predicates and precond not in constant_predicates:
                break
        actions.append(action)
        print(action)
    return actions


actions = get_actions(initial_predicates, constant_predicates)

(:action move
    :parameters (?pos1 ?pos2 )
    :precondition (and (adjacent ?pos1 ?pos2) (at ?pos1) (not (wall ?pos2)) (not (obstacle ?pos2)))
    :effect (and (at ?pos2) (not (at ?pos1)))
)
(:action shoot
    :parameters (?pos1 ?pos2 )
    :precondition (and (adjacent ?pos1 ?pos2) (at ?pos1) (not (wall ?pos2)) (munition))
    :effect (and (not (obstacle ?pos2)) (not (munition)))
)
